In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [3]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [5]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

In [28]:
with open('password.txt') as f:
    password = f.read()

In [29]:
email = 'minu7972'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [30]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [31]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [32]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

In [33]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [34]:
driver.get(url)
time.sleep(5)

In [35]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [36]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [37]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'#리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다!!..........................................@gunwoo_o94 님의 멋진 사진 너무 감사합니다♥️역시 찬 바람 불 땐 스튜🙃오늘도 변함없이 맛있게 준비해서 11시에 오픈합니다🙆\u200d♀️전 메뉴 포장 가능하며,도착 30분 전 전화주시면 시간 맞춰 포장 도와드리겠습니다🙋\u200d♀️오늘도 #안녕하샌'

In [38]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#리그램', '#안녕하샌']

In [39]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-07'

In [40]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'328'

In [41]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

'월정리해변'

In [42]:
row = [content, date, like, place, tags]

In [43]:
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
time.sleep(2)

In [44]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [45]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [46]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [47]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button/div/span').click()
        time.sleep(5)
    except:
        time.sleep(5)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(5)

  0%|          | 0/500 [00:00<?, ?it/s]

In [65]:
df = pd.DataFrame(results)
df

,0,1,2,3,4
0,#리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다...,2021-12-07,328,월정리해변,"[#리그램, #안녕하샌]"
1,"아침부터 스튜 포장이 많네요🙋‍♀️스튜는 마늘빵과 함께 제공해드리고 있으며,마늘빵 ...",2021-12-08,440,월정리해변,[#안녕하샌]
2,.🍺삼도동 집아페🍺나만 알고 싶은 아담한 로컬 이자카야 찐맛집둘이 가도 좋고 혼술 ...,2021-12-06,"1,180",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
3,항상 멋진 오션뷰와 맛있는 음식애월하미에 한번 오시면 재방문의사 200%!!!!! ...,2021-12-08,415,애월하미,"[#천상의바다, #애월북서길54, #애월횟집, #애월읍횟집, #제주도횟집, #제주도..."
4,🎀 제주에 진짜 맛집 🎀⁣⁣문개항아리 (づ￣ 3￣)づ⁣⁣⁣직접 잡아오는 제주도 자연...,2021-12-07,0,,"[#제주도맛집, #문어라면, #애월해물라면, #애월해산물맛집, #제주애월맛집추천, ..."
...,...,...,...,...,...
467,항상찍는자리,2021-12-08,186,정신없는 제주공항 어딘가,[]
468,오늘도 날씨 맑음 수요일 퐁당카레 시작했습니다!!.🏖#퐁당카레🏖.⏰점심영업 AM 1...,2021-12-08,20,퐁당카레,[#퐁당카레🏖.⏰점심영업]
469,예전에 맛있게 먹은 기억때문에 또 재방문했어요.입맛 까다로운 부모님 모시고 왔는데 ...,2021-12-08,161,,"[#색달식당중문본점*제주, #색달식당중문본점, #제주도갈치조림, #제주도맛집, #서..."
470,여기는 노형수퍼마켙이라는곳인데마켓 시옷받침이 아니라서 주소검색하기 헷갈렸음.아르떼...,2021-12-08,35,제주 노형수퍼마켙,"[#제주여행, #제주카라반노지캠, #제주공항, #제주숙소, #제주핫플, #카라반라이..."


In [67]:
df.drop_duplicates(subset=0,inplace=True)
df

,0,1,2,3,4
0,#리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다...,2021-12-07,328,월정리해변,"[#리그램, #안녕하샌]"
1,"아침부터 스튜 포장이 많네요🙋‍♀️스튜는 마늘빵과 함께 제공해드리고 있으며,마늘빵 ...",2021-12-08,440,월정리해변,[#안녕하샌]
2,.🍺삼도동 집아페🍺나만 알고 싶은 아담한 로컬 이자카야 찐맛집둘이 가도 좋고 혼술 ...,2021-12-06,"1,180",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
3,항상 멋진 오션뷰와 맛있는 음식애월하미에 한번 오시면 재방문의사 200%!!!!! ...,2021-12-08,415,애월하미,"[#천상의바다, #애월북서길54, #애월횟집, #애월읍횟집, #제주도횟집, #제주도..."
4,🎀 제주에 진짜 맛집 🎀⁣⁣문개항아리 (づ￣ 3￣)づ⁣⁣⁣직접 잡아오는 제주도 자연...,2021-12-07,0,,"[#제주도맛집, #문어라면, #애월해물라면, #애월해산물맛집, #제주애월맛집추천, ..."
...,...,...,...,...,...
467,항상찍는자리,2021-12-08,186,정신없는 제주공항 어딘가,[]
468,오늘도 날씨 맑음 수요일 퐁당카레 시작했습니다!!.🏖#퐁당카레🏖.⏰점심영업 AM 1...,2021-12-08,20,퐁당카레,[#퐁당카레🏖.⏰점심영업]
469,예전에 맛있게 먹은 기억때문에 또 재방문했어요.입맛 까다로운 부모님 모시고 왔는데 ...,2021-12-08,161,,"[#색달식당중문본점*제주, #색달식당중문본점, #제주도갈치조림, #제주도맛집, #서..."
470,여기는 노형수퍼마켙이라는곳인데마켓 시옷받침이 아니라서 주소검색하기 헷갈렸음.아르떼...,2021-12-08,35,제주 노형수퍼마켙,"[#제주여행, #제주카라반노지캠, #제주공항, #제주숙소, #제주핫플, #카라반라이..."


In [ ]:
df[3].isin('')

In [92]:
df.columns = ['내용','날짜','조회수','장소','테그']

In [93]:
df.columns

Index(['내용', '날짜', '조회수', '장소', '테그'], dtype='object')

In [102]:
df.to_csv('인스타그램 맛집.csv',index=False)

In [6]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=96.0.4664.93)
